# Binder Test
This notebook will demonstrate a use of mybinder with PyBigDFT. First, you will need to create a repository on [github](http://www.github.com). In this repository, you should put the following `requirements.txt` file.

In [ ]:
! cat requirements.txt

You will next want to compile BigDFT on your local computer. Then, from `install/lib/python*/site-packages/` you should copy the `BigDFT` and `futile` directories.

In [ ]:
! ls BigDFT

In [ ]:
! ls futile

## Workflow Test
Now let's test this out with a calculation. First setup a simple system. Notice how we strategically cache results so that my binder doesn't have to do any heavy lifting.

In [ ]:
from os import system
from os.path import exists

if not exists("1uao.pdb"):
    system("pdbfixer --pdbid=1uao --add-atoms=all --output=1uao.pdb --water-box 2 2 2")

In [ ]:
if not exists("minimized.pdb"):
    from simtk.openmm.app import *
    from simtk.openmm import *
    from simtk.unit import *
    from sys import stdout

    pdb = PDBFile('1uao.pdb')
    forcefield = ForceField('amber99sb.xml', 'tip3p.xml')
    system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
                                     nonbondedCutoff=1*nanometer, constraints=HBonds)
    integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
    simulation = Simulation(pdb.topology, system, integrator)
    simulation.context.setPositions(pdb.positions)
    simulation.minimizeEnergy()
    state = simulation.context.getState(getPositions=True)
    with open("minimized.pdb", "w") as ofile:
        PDBFile.writeFile(pdb.topology, state.getPositions(), ofile)

We read in the created system and visualize.

In [ ]:
from BigDFT.IO import read_pdb
from BigDFT.UnitCells import UnitCell
with open("minimized.pdb") as ifile:
    sys = read_pdb(ifile)
# Note that pdbfixer generates a system which is periodic by default
sys.cell = UnitCell()

In [ ]:
# from BigDFT.Visualization import InlineVisualizer
# from BigDFT.Systems import System

# vis_sys = System()
# for fragid in sys:
#     if fragid[:3] != "HOH" and fragid[:3] != " NA":
#         vis_sys[fragid] = sys[fragid]

# viz = InlineVisualizer(400, 300)
# viz.display_system(vis_sys)

Run the calculation.

In [ ]:
from BigDFT.Inputfiles import Inputfile
inp = Inputfile()
inp.set_xc("PBE")
inp.set_hgrid(0.4)
# Default Na has semi-core states
inp.set_psp_file(filename="psppar.Na", element="Na")
inp["import"] = "linear"

In [ ]:
from BigDFT.Calculators import SystemCalculator
code = SystemCalculator(skip=True)

In [ ]:
log = code.run(posinp=sys.get_posinp(), input=inp, name="test", run_dir="work")

In [ ]:
print(log.energy)

In [ ]:
from BigDFT.PostProcessing import BigDFTool
tool = BigDFTool()

In [ ]:
sys.set_logfile_info(log)

In [ ]:
pv = tool.run_compute_purity(sys, log)

Do some basic post processing.

In [ ]:
def plot(protein, water, ylabel):
    from BigDFT.Systems import plot_fragment_information
    from matplotlib import pyplot as plt
    from numpy.linalg import norm
    from numpy import log

    fig, axs = plt.subplots(1,2, figsize=(12,3))
    plot_fragment_information(axs[0], protein)
    axs[0].set_title("Protein + Ions", fontsize=12)
    axs[0].set_ylabel(ylabel)

    axs[1].set_title("Water", fontsize=12)
    axs[1].plot(sorted(water.values()), 'kx--')
    axs[1].set_xlabel("Molecule", fontsize=12)

    fig.tight_layout()

In [ ]:
import ipywidgets as widgets

w = widgets.Dropdown(options=['Charge', 'Dipole', "Purity"],
                     value='Purity',
                     description='Property:',
                     disabled=False)
display(w)

In [ ]:
if w.value == "Charge":
    protein = {x: y.qcharge() - y.nel for x, y in sys.items() if "HOH" not in x}
    water = {x: y.qcharge() - y.nel for x, y in sys.items() if "HOH" in x}
    ylabel = "Charge (A.U.)"
elif w.value == "Dipole":
    from numpy.linalg import norm
    protein = {x: norm(y.q0) for x, y in sys.items() if "HOH" not in x}
    water = {x: norm(y.q0) for x, y in sys.items() if "HOH" in x}
    ylabel = "Dipole (A.U.)"
elif w.value == "Purity":
    protein = {x: pv[x] for x, y in sys.items() if "HOH" not in x}
    water = {x: pv[x] for x, y in sys.items() if "HOH" in x}
    ylabel = "Purity"

In [ ]:
plot(protein, water, ylabel)